# Prosumer Behavior Model Training Pipeline and Prediction

## Setup

In [0]:
%pip install -U mlflow

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
%pip show mlflow

Name: mlflow
Version: 3.7.0
Summary: MLflow is an open source platform for the complete machine learning lifecycle
Home-page: https://mlflow.org
Author: 
Author-email: 
License: Copyright 2018 Databricks, Inc.  All rights reserved.
        
                                        Apache License
                                   Version 2.0, January 2004
                                http://www.apache.org/licenses/
        
           TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION
        
           1. Definitions.
        
              "License" shall mean the terms and conditions for use, reproduction,
              and distribution as defined by Sections 1 through 9 of this document.
        
              "Licensor" shall mean the copyright owner or entity authorized by
              the copyright owner that is granting the License.
        
              "Legal Entity" shall mean the union of the acting entity and all
              other entities that control, ar

In [0]:
import mlflow
#use databricks UC registry
mlflow.set_registry_uri("databricks-uc")

In [0]:
import re
userName = spark.sql("SELECT CURRENT_USER").collect()[0]['current_user()']
userName0 = userName.split("@")[0]
userName0 = re.sub(r"[!#$%&'*+/=?^`{}|.\\-]+", '_', userName0)
userName1 = userName.split("@")[1]
userName = f'{userName0}@{userName1}'
userNameReg = re.sub(r'[@\.]', '_', userName)

#Needed for MLFlow and Serverless
import os
os.environ['SPARKML_TEMP_DFS_PATH'] = '/Volumes/cscie103_catalog_final/gold/data'
os.environ['MLFLOW_DFS_TMP'] = '/Volumes/cscie103_catalog_final/gold/data'

spark.sql('CREATE CATALOG IF NOT EXISTS cscie103_catalog_final');
spark.sql('USE CATALOG cscie103_catalog_final')

spark.sql('CREATE SCHEMA IF NOT EXISTS gold');
spark.sql('USE cscie103_catalog_final.gold')

spark.sql(f'CREATE VOLUME if not exists cscie103_catalog_final.gold.data')

userDir = f"/Volumes/cscie103_catalog_final/gold/data"
data_path_json = f"/Volumes/cscie103_catalog_final/gold/data/json"
data_path_csv = f"/Volumes/cscie103_catalog_final/gold/data/csv"
data_path = data_path_csv
dbutils.fs.mkdirs(f"{data_path}")
dbutils.fs.rm('/Volumes/cscie103_catalog_final/gold/data/checkpoint', True)
spark.sql('DROP TABLE IF EXISTS cscie103_catalog_final.gold.streaming_predictions_tbl')

print('UserDir ' + userDir)
print('userName '+ userName)
print('Database gold')

spark.sql(f"use cscie103_catalog_final.gold")

UserDir /Volumes/cscie103_catalog_final/gold/data
userName lil658@g.harvard.edu
Database gold


DataFrame[]

## Score on streaming data

In [0]:
# Load our features dataset

df = spark.sql(f'''select county_id, county_name, latitude, longitude
,(case when is_business then 'Business' else 'Consumer' end) AS eic_type
,(case when is_consumption then 'Consumption' else 'Production' end) AS target_type
,(case when t_observ_4hour=0 then '00-04'
    when t_observ_4hour=1 then '04-08' 
    when t_observ_4hour=2 then '08-12' 
    when t_observ_4hour=3 then '12-16' 
    when t_observ_4hour=4 then '16-20' 
    when t_observ_4hour=5 then '20-24' end) AS time_of_day,
product_type,
 data_block_id,
 wh_temperature,
 wh_direct_solar_radiation,
 target from cscie103_catalog_final.gold.county_energy_weather_4hours_mv''')
df = df.repartition('county_id')

display(df.head(10))

county_id,county_name,latitude,longitude,eic_type,target_type,time_of_day,product_type,data_block_id,wh_temperature,wh_direct_solar_radiation,target
15,VÕRUMAA,57.779999999999994,27.1,Business,Production,08-12,Fixed,234,8.979999999999999,300.26666666666665,254.901
15,VÕRUMAA,57.779999999999994,27.1,Consumer,Consumption,08-12,Spot,96,-11.18,1.8,164.061
15,VÕRUMAA,57.779999999999994,27.1,Business,Production,12-16,Spot,445,-2.8,4.7,4.182
15,VÕRUMAA,57.779999999999994,27.1,Business,Production,08-12,Fixed,598,10.186666666666666,285.93333333333334,71.851
15,VÕRUMAA,57.779999999999994,27.1,Consumer,Consumption,04-08,Fixed,251,2.6849999999999996,56.9,33.842
15,VÕRUMAA,57.779999999999994,27.1,Business,Production,20-24,Spot,184,-0.825,0.0,0.0
15,VÕRUMAA,57.779999999999994,27.1,Consumer,Production,20-24,Fixed,179,0.72,0.0,0.0
15,VÕRUMAA,57.779999999999994,27.1,Business,Consumption,08-12,Spot,186,-7.126666666666667,101.73333333333333,246.182
15,VÕRUMAA,57.779999999999994,27.1,Consumer,Production,04-08,Fixed,29,7.5550000000000015,0.25,0.0
15,VÕRUMAA,57.779999999999994,27.1,Consumer,Consumption,16-20,Spot,274,14.459999999999999,112.2,33.577


In [0]:
# Only sample 20% of the data for stream app demo
fp_train_df, fp_test_df = df.randomSplit([0.95, 0.05], seed=42)

In [0]:
# simulate streaming data source
# repartition so we get a bunch of files which helps with extended streaming
# Out the csv with header
streamingData = fp_test_df.repartition(500)
streamingData.write.mode("overwrite").option("header", "true").csv(data_path)

## Load a County Model from mlflow

* County: 0
Final Project Train rows: 198229 Final Project Test rows: 49319
Test RMSE: 1733.095094770264

 > * 'cscie103_catalog_final.gold.fp-model-0': https://dbc-bf0c22bc-3661.cloud.databricks.com/explore/data/models/cscie103_catalog_final/gold/fp-model-0/version/1?o=3544939500018450

* County: 1
Final Project Train rows: 85264 Final Project Test rows: 21548
Test RMSE: 50.792517636787956

 > * 'cscie103_catalog_final.gold.fp-model-1': https://dbc-bf0c22bc-3661.cloud.databricks.com/explore/data/models/cscie103_catalog_final/gold/fp-model-1/version/1?o=3544939500018450
 
* County: 2
Final Project Train rows: 106998 Final Project Test rows: 26908
Test RMSE: 299.84007294177604

 > * 'cscie103_catalog_final.gold.fp-model-2': https://dbc-bf0c22bc-3661.cloud.databricks.com/explore/data/models/cscie103_catalog_final/gold/fp-model-2/version/1?o=3544939500018450

* County: 6
Final Project Train rows: 28392 Final Project Test rows: 7212
Test RMSE: 82.67250299226838

* County: 7
Final Project Train rows: 160629 Final Project Test rows: 40007
Test RMSE: 262.99752475029146

> * 'cscie103_catalog_final.gold.fp-model-7': https://dbc-bf0c22bc-3661.cloud.databricks.com/explore/data/models/cscie103_catalog_final/gold/fp-model-7/version/1?o=3544939500018450

* County: 8
Final Project Train rows: 85264 Final Project Test rows: 21548
Test RMSE: 136.15668810452323

> * 'cscie103_catalog_final.gold.fp-model-8': https://dbc-bf0c22bc-3661.cloud.databricks.com/explore/data/models/cscie103_catalog_final/gold/fp-model-8/version/1?o=3544939500018450

* County: 4
Final Project Train rows: 136879 Final Project Test rows: 34311
Test RMSE: 198.75035644768352

* County: 11
Final Project Train rows: 183526 Final Project Test rows: 45772
Test RMSE: 1042.838145532865


In [0]:
from mlflow.tracking.client import MlflowClient
client = MlflowClient()

In [0]:
import traceback
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository

def print_registered_model_info(rm):
    #print("name: {}".format(rm.name))
    #print("tags: {}".format(rm.tags))
    #print("description: {}".format(rm.description))
    print(rm)
    print('----------')

def print_model_version_info(mv):
    #print("Name: {}".format(mv.name))
    #print("Version: {}".format(mv.version))
    #print("Tags: {}".format(mv.tags))    
    print(mv)
    print('----------')

def print_models_info(mv):
    for m in mv:
        print(m)
    print('-----------')
        #print("name: {}".format(m.name))
        #print("latest version: {}".format(m.version))
        #print("run_id: {}".format(m.run_id))
        #print("current_stage: {}".format(m.current_stage))    

In [0]:
# Load the model from mlflow
model_name_stem = f"fp-model"
model_directory = {}

# loop through the county_id and load the model into mode_directory
for county_id in df.select('county_id').distinct().collect():
  model_name = f"{model_name_stem}-{county_id.county_id}"
  client.set_registered_model_alias(model_name, 'init', '1')
  county_model = client.get_registered_model(model_name)
  county_model_uri =f"models:/{model_name}@init"
  model_directory[county_id.county_id] = county_model_uri
  print(f"County {county_id.county_id} model {model_name} loaded")
  print_registered_model_info(model_directory[county_id.county_id])


County 15 model fp-model-15 loaded
models:/fp-model-15@init
----------
County 1 model fp-model-1 loaded
models:/fp-model-1@init
----------
County 4 model fp-model-4 loaded
models:/fp-model-4@init
----------
County 13 model fp-model-13 loaded
models:/fp-model-13@init
----------
County 8 model fp-model-8 loaded
models:/fp-model-8@init
----------
County 6 model fp-model-6 loaded
models:/fp-model-6@init
----------
County 3 model fp-model-3 loaded
models:/fp-model-3@init
----------
County 11 model fp-model-11 loaded
models:/fp-model-11@init
----------
County 0 model fp-model-0 loaded
models:/fp-model-0@init
----------
County 2 model fp-model-2 loaded
models:/fp-model-2@init
----------
County 7 model fp-model-7 loaded
models:/fp-model-7@init
----------
County 10 model fp-model-10 loaded
models:/fp-model-10@init
----------
County 9 model fp-model-9 loaded
models:/fp-model-9@init
----------
County 14 model fp-model-14 loaded
models:/fp-model-14@init
----------
County 5 model fp-model-5 loaded


In [0]:
from pyspark.sql.types import StructType

schema = streamingData.schema
inputStreamDF = spark \
  .readStream \
  .schema(schema) \
  .option("maxFilesPerTrigger", 2) \
  .csv(data_path)

#loop through the inputStreamDF by county_id and score the data and union all the data
inputStreamDF.writeStream.outputMode("Append") \
.format("delta") \
.trigger(availableNow=True) \
.option("checkpointLocation", f"{userDir}/stream_predictions_checkpoint/") \
.toTable("stream_input")


In [0]:
# county map lookup table
mapTbl = spark.sql("select * from county_map").dropna()
display(mapTbl.head(10))
countyTbl = mapTbl.select('county_id','county_name', 'latitude', 'longitude').distinct()
countyTbl=countyTbl.dropna()
display(countyTbl.head(10))

inputStream = spark.readStream.table("stream_input").dropna()

for county in countyTbl.collect():

  # get the model
  cid = county[0]
  cname = county[1]
  clat = county[2]
  clon = county[3]
  model_name = f"{model_name_stem}-{cid}"
  print(f"County {cid} {cname} model {model_name} loaded")

  if cid in (5,14, 0,2,1,5,6,7,8,9,10,11,13,15):
    print(f"County {cid} {cname} model {model_name} skipped")
    continue

  county_model =mlflow.pyfunc.load_model(model_directory[cid])
  
  #county_model = mlflow.pyfunc.spark_udf(spark, model_uri=county_model.model_uri)
  base_model = county_model.get_raw_model()
  # score the input stream
  scoredStream = base_model.transform(inputStream.filter(inputStream.county_id == cid))
  # write to stream                                      

  scoredStream.createOrReplaceTempView(f"temp_predictions_{cid}")

  # create the prediction summary
  predict_summary_stream = spark.sql(f'''SELECT county_id, county_name, latitude, longitude, eic_type, target_type, count(*) as qt FROM temp_predictions_{cid} GROUP BY county_id, county_name, latitude, longitude, eic_type, target_type''')

  # save the prediction summary to delta table
  predict_summary_stream.writeStream \
    .outputMode("complete") \
    .trigger(availableNow=True) \
    .format("delta") \
    .option("checkpointLocation", f"{userDir}/stream_predictions_sum_checkpoint/") \
    .toTable(f"stream_predictions_summary").awaitTermination()

  # save output to delta table
  scoredStream.writeStream.outputMode("Append") \
  .option("checkpointLocation", f"{userDir}/stream_predictions_output_checkpoint/") \
  .format("delta") \
  .trigger(availableNow=True) \
  .toTable(f"stream_predictions_{cid}").awaitTermination()

     

county_id,county_name,station_count,longitude,latitude
2,IDA-VIRUMAA,4,27.45,59.249999999999986
5,LÄÄNE-VIRUMAA,4,26.45,59.249999999999986
4,JÕGEVAMAA,3,26.366666666666664,58.69999999999999
13,VALGAMAA,1,26.2,57.9
10,SAAREMAA,4,22.575,58.425
1,HIIUMAA,1,22.7,58.79999999999999
6,LÄÄNEMAA,2,23.7,58.94999999999999
14,VILJANDIMAA,3,25.53333333333333,58.4
0,HARJUMAA,6,24.866666666666664,59.29999999999998
9,RAPLAMAA,3,24.53333333333333,58.899999999999984


county_id,county_name,latitude,longitude
2,IDA-VIRUMAA,59.249999999999986,27.45
5,LÄÄNE-VIRUMAA,59.249999999999986,26.45
4,JÕGEVAMAA,58.69999999999999,26.366666666666664
13,VALGAMAA,57.9,26.2
10,SAAREMAA,58.425,22.575
1,HIIUMAA,58.79999999999999,22.7
6,LÄÄNEMAA,58.94999999999999,23.7
14,VILJANDIMAA,58.4,25.53333333333333
0,HARJUMAA,59.29999999999998,24.866666666666664
9,RAPLAMAA,58.899999999999984,24.53333333333333


County 2 IDA-VIRUMAA model fp-model-2 loaded
County 2 IDA-VIRUMAA model fp-model-2 skipped
County 5 LÄÄNE-VIRUMAA model fp-model-5 loaded
County 5 LÄÄNE-VIRUMAA model fp-model-5 skipped
County 4 JÕGEVAMAA model fp-model-4 loaded
County 13 VALGAMAA model fp-model-13 loaded
County 13 VALGAMAA model fp-model-13 skipped
County 10 SAAREMAA model fp-model-10 loaded
County 10 SAAREMAA model fp-model-10 skipped
County 1 HIIUMAA model fp-model-1 loaded
County 1 HIIUMAA model fp-model-1 skipped
County 6 LÄÄNEMAA model fp-model-6 loaded
County 6 LÄÄNEMAA model fp-model-6 skipped
County 14 VILJANDIMAA model fp-model-14 loaded
County 14 VILJANDIMAA model fp-model-14 skipped
County 0 HARJUMAA model fp-model-0 loaded
County 0 HARJUMAA model fp-model-0 skipped
County 9 RAPLAMAA model fp-model-9 loaded
County 9 RAPLAMAA model fp-model-9 skipped
County 15 VÕRUMAA model fp-model-15 loaded
County 15 VÕRUMAA model fp-model-15 skipped
County 3 JÄRVAMAA model fp-model-3 loaded
County 7 PÄRNUMAA model fp-mode

In [0]:
%sql
--drop table if exists cscie103_catalog_final.gold.stream_predictions_11;
select count(*) from cscie103_catalog_final.gold.stream_input
union all
select count(*) from cscie103_catalog_final.gold.stream_predictions_2
union all
select count(*) from cscie103_catalog_final.gold.stream_predictions_summary;

### Q: display count of the predicted labels
* Use pyspark streaming on temporary table, to get `prediction` and `count` of prediction
* use writeStream with outputMode=complete, trigger=availableNow to table streaming_predictions_tbl


* Run SQL a few times and see if counts are refreshing

In [0]:
%sql 
select * from cscie103_catalog_final.gold.stream_predictions_summary

In [0]:
for s in spark.streams.active:
  s.stop()  